# **NOTEBOOK DE CALCUL DES EMISSIONS SONORES**

In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import math
import Emission

In [28]:
testBase=Emission.Route(12000,100,90,80)
testDouble=Emission.Route(20000,150,90,80)

In [29]:
NiveBase, NiveDouble=testBase.lwm, testDouble.lwm

In [30]:
NiveBase, NiveDouble

(96.62674670416406, 98.82348541391119)